## Imports, data load, metric function definition

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import BallTree

In [92]:
X_train = np.load('X_train_surge.npz')
Y_train = pd.read_csv('Y_train_surge.csv')
X_test = np.load('X_test_surge.npz')

In [47]:
t_slp = X_train['t_slp'] / 3600
t_slp_delta = t_slp - t_slp[:, 0].reshape(-1, 1)
np.allclose(np.round(t_slp_delta), np.round(t_slp_delta)[0])

True

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

rel_timestamps = np.arange(24*5+1, step=3)[:-1]
t_slp_normalisation = 1e5

def preprocessing(X, device, slp_mean=None, slp_std=None, t_slp_mean=None, t_slp_std=None,
                 surge_mean=None, surge_std=None):
    def normalised_tensor(array, mean, std):
        return torch.from_numpy((array - mean) / std).to(device)
    
    slp = X['slp'].reshape(-1, 40, 41, 41)
    slp = np.roll(slp, shift=-11, axis=3)
    if slp_mean is None:
        slp_mean = np.mean(slp)
    if slp_std is None:
        slp_std = np.std(slp)
    slp = normalised_tensor(slp, slp_mean, slp_std)
    
    fst_slp = X['t_slp'][:, 0] / 3600
    fst_slp_tmp = fst_slp.reshape(-1, 1)
    
    def rel_surge_time(index):
        t_surge = X[index] / 3600
        return torch.from_numpy(t_surge - fst_slp_tmp).to(device)

    t_surge1_in = rel_surge_time('t_surge1_input')
    t_surge2_in = rel_surge_time('t_surge2_input')
    t_surge1_out = rel_surge_time('t_surge1_output')
    t_surge2_out = rel_surge_time('t_surge2_output')
    
    if t_slp_mean is None:
        t_slp_mean = np.mean(fst_slp)
    if t_slp_std is None:
        t_slp_std = np.std(fst_slp)
    fst_slp = normalised_tensor(fst_slp, t_slp_mean, t_slp_std)
    
    surge1 = X['surge1_input']
    surge2 = X['surge2_input']
    if surge_mean is None or surge_std is None:
        surges = np.concatenate([surge1, surge2], axis=None)
        surge_mean = np.mean(surges)
        surge_std = np.std(surges)
    surge1_in = normalised_tensor(surge1, surge_mean, surge_std)
    surge2_in = normalised_tensor(surge2, surge_mean, surge_std)
    
    return X['id_sequence'], slp, slp_mean, slp_std, \
            fst_slp, t_slp_mean, t_slp_std, \
            t_surge1_in, t_surge2_in, t_surge1_out, t_surge2_out, \
            surge1_in, surge2_in, surge_mean, surge_std

In [94]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [95]:
train_id_seq, train_slp, slp_mean, slp_std, \
train_fst_slp, t_slp_mean, t_slp_std, \
train_t_surge1_in, train_t_surge2_in, train_t_surge1_out, train_t_surge2_out, \
train_surge1_in, train_surge2_in, surge_mean, surge_std = preprocessing(X_train, device)

In [96]:
test_id_seq, test_slp, _, _, \
test_fst_slp, _, _, \
test_t_surge1_in, test_t_surge2_in, test_t_surge1_out, test_t_surge2_out, \
test_surge1_in, test_surge2_in, _, _ = preprocessing(X_test, device, slp_mean, slp_std, t_slp_mean, t_slp_std, surge_mean, surge_std)

In [79]:
from torch.utils.data import TensorDataset, DataLoader

resnet_versions = {
    18 : models.resnet18,
    34 : models.resnet34,
    50 : models.resnet50,
}

batch_size = 64

# TODO: shuffle train data
train_dataset = TensorDataset(
    train_slp,
#     train_fst_slp,
    train_t_surge1_in,
    train_surge1_in,
    train_t_surge2_in,
    train_surge2_in,
    train_t_surge1_out,
    train_t_surge2_out,
#     train_surge1_out, # TODO: add this from Y_train (use surge_mean, surge_std to normalize the tide heights)
#     train_surge2_out  # TODO: do the same for the test dataset
)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

test_dataset = TensorDataset(
    test_slp,
#     test_fst_slp,
    test_t_surge1_in,
    test_surge1_in,
    test_t_surge2_in,
    test_surge2_in,
    test_t_surge1_out,
    test_t_surge2_out
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class Conv2d(nn.Conv2d):
    """Weight standardisation
    (see https://arxiv.org/pdf/1903.10520.pdf and https://github.com/joe-siyuan-qiao/WeightStandardization)
    """

    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1, bias=True):
        super(Conv2d, self).__init__(in_channels, out_channels, kernel_size, stride,
                 padding, dilation, groups, bias)

    def forward(self, x):
        weight = self.weight
        weight_mean = weight.mean(dim=1, keepdim=True).mean(dim=2,
                                  keepdim=True).mean(dim=3, keepdim=True)
        weight = weight - weight_mean
        std = weight.view(weight.size(0), -1).std(dim=1).view(-1, 1, 1, 1) + 1e-5
        weight = weight / std.expand_as(weight)
        return F.conv2d(x, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
    
class Network(nn.Module):
    
    def __init__(self, resnet_layers, out_features, back_window):
        self.resnet = resnet_versions[resnet_layers]()
        
        # Change the number of out_features from 1000 to `out_features`
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, out_features)
        lstm_input_size = out_features + 
        self.lstm = nn.LSTM(input_size=lstm_input_size, hidden_size=, num_layers=1, bias=True, batch_first=True, \
                            dropout=.2, bidirectional=False, proj_size=0)
        # LSTM:
        # Add prediction time?
        # Learn to associate pressure_mapt to their timestamps
        
    def forward(self, slp, t_surge1, surge1, t_surge2, surge2):
        slp_embeddings self.resnet(slp.unsqueeze(1))
        torch.cat([slp_embeddings, ], dim=1)
        self.lstm()
        
        pass

In [101]:
loss_weights = torch.linspace(1, 0.1, 10, requires_grad=False)
    
def surge_prediction_metric(surge1_true, surge_2_true, surge1_pred, surge2_pred):
    surge1_score = torch.matmul(torch.square(surge1_true - surge1_pred), loss_weights).mean(dim=1)
    surge2_score = torch.matmul(torch.square(surge2_true - surge2_pred), loss_weights).mean(dim=1)

    return surge1_score + surge2_score

In [ ]:
def train(model, loss_fn, optmiser, epochs):
    train_loss = []
    train_acc = []
    
    for epoch_num in range(epochs):
        model.train()
        running_loss = []

        for batch, (slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out, surge1_out, surge2_out) in enumerate(train_dataloader):
            slp = slp.to(device)
            # ...
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            running_loss.append(loss.item())
            
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            
        epoch_loss = np.mean(running_loss)
        train_loss.append(epoch_loss)
        print(f'Epoch {epoch_num+1:03d} | Loss: {epoch_loss:.4f}')
        
        if epoch_num % 5 == 0:
            test_loss = evaluate(model, loss_fn)
            print(f'\tTest loss: {test_loss:.4f}')
        
    return train_loss

def evaluate(model, loss_fn):
    with torch.no_grad():
        model.eval()
        losses = []
        for slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out, surge1_out, surge2_out in enumerate(test_dataloader):
            slp = slp.to(device)
            # ...
            surge2_out = surge2_out.to(device)
            surge1_out_pred, surge2_out_pred = model(slp, t_surge1_in, surge1_in, t_surge2_in, surge2_in, t_surge1_out, t_surge2_out)
            
            loss = loss_fn(surge1_out, surge2_out, surge1_out_pred, surge2_out_pred)
            losses.append(loss.item())
        return np.mean(losses)

In [4]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

In [ ]:
model = Network(...)
optimiser = optim.Adam(model.parameters())
_ = train(model, surge_prediction_metric, optmiser, epochs=100)

## Benchmark
Train using kNN of pressure fields at two instants in time, with 40 neighbours

In [5]:
nfields = 2; time_step_slp = 8
slp_train = []
slp_all = X_train['slp']
for i in range(5559):
    slp_train.append(np.ndarray.flatten(slp_all[i,-1]))
    for j in range(1,nfields):
        slp_train[-1] = np.concatenate( ( slp_train[-1], np.ndarray.flatten(slp_all[i,-1-j*time_step_slp]) ) )
slp_train = np.array(slp_train)

In [6]:
slp_test = []
slp_all_test = X_test['slp']
for i in range(509):
    slp_test.append(np.ndarray.flatten(slp_all_test[i,-1]))
    for j in range(1,nfields):
        slp_test[-1] = np.concatenate( ( slp_test[-1], np.ndarray.flatten(slp_all_test[i,-1-j*time_step_slp]) ) )
slp_test = np.array(slp_test)

In [7]:
tree = BallTree(slp_train)

In [8]:
surge_test_benchmark = []; k = 40
for i in range(509):
    dist, ind = tree.query([slp_test[i]], k=k)
    surge_test_benchmark.append(np.mean(surge_train[ind[0]], axis=0))
surge_test_benchmark = np.array(surge_test_benchmark)

In [9]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=surge_test_benchmark, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('Y_test_benchmark.csv', index_label='id_sequence', sep=',')